## Charger les données

In [1]:
import geopandas as gpd

# Charger les données des stations de vélo et des quartiers en geosjson
stations = gpd.read_file('data/paris_stations_velo.geojson')
quartiers = gpd.read_file('data/paris_quartiers.geojson')

# affichier les données
print(stations.head())
print(quartiers.head())

                                  name  bike_stands  available_bikes  \
0               14002 - RASPAIL QUINET           44                4   
1  20503 - COURS DE VINCENNES PYRÉNÉES           21                3   
2             20011 - PYRÉNÉES-DAGORNO           21                0   
3        31008 - VINCENNES (MONTREUIL)           56                0   
4          43006 - MINIMES (VINCENNES)           28               27   

                   geometry  
0   POINT (2.32956 48.8392)  
1  POINT (2.40591 48.84772)  
2   POINT (2.40517 48.8555)  
3   POINT (2.43737 48.8577)  
4  POINT (2.43079 48.84149)  
   id           district_name  population  \
0   1  St-Germain-l'Auxerrois        1672   
1   2                  Halles        8984   
2   3            Palais-Royal        3195   
3   4           Place-Vendôme        3044   
4   5                 Gaillon        1345   

                                            geometry  
0  POLYGON ((2.34459 48.85405, 2.34459 48.85405, ...  
1  POL

## requete 1 : afficher les stations d'une zone de Paris Centre

In [2]:
from shapely.geometry import Point
import folium

# Fonction pour trouver les stations de vélo dans un rayon donné autour d'un point
def find_stations_within_radius(data, center, radius):
    buffer = center.buffer(radius/1000) # convertir le rayon en km
    stations_within = data[data.geometry.within(buffer)]
    return stations_within

# Définir un point central (centre de Paris) et un rayon
latitude = 48.8566
longitude = 2.3522
radius = 500  # mètres
center_point = Point(longitude, latitude)

# Trouver les stations dans le rayon
stations_in_radius = find_stations_within_radius(stations, center_point, radius)

# Afficher les stations trouvées
print(stations_in_radius)

# CRéer la carte folium
m = folium.Map(location=[latitude, longitude], zoom_start=14)
folium.Circle([latitude, longitude], radius=radius, color='blue', fill=True).add_to(m)

# Ajouter uniquement les stations dans le rayon
for idx, row in stations_in_radius.iterrows():
    folium.Marker([row.geometry.y, row.geometry.x], popup=row['name']).add_to(m)

# Afficher la carte
m


                                        name  bike_stands  available_bikes  \
0                     14002 - RASPAIL QUINET           44                4   
1        20503 - COURS DE VINCENNES PYRÉNÉES           21                3   
2                   20011 - PYRÉNÉES-DAGORNO           21                0   
3              31008 - VINCENNES (MONTREUIL)           56                0   
4                43006 - MINIMES (VINCENNES)           28               27   
...                                      ...          ...              ...   
1221                       12011 - CHARENTON           36               35   
1222             13110 - MOULIN DE LA POINTE           22                4   
1223                   07006 - QUAI VOLTAIRE           32               25   
1224  21311 - GAMBETTA (ISSY LES MOULINEAUX)           55               47   
1225                 01020 - RIVOLI CONCORDE           40               37   

                      geometry  
0      POINT (2.32956 48.8392)

## requete 2 : afficher les données du quartier de Javel

In [3]:

# Fonction pour trouver les stations de vélo dans un quartier donné
def find_stations_in_quartier(stations, quartier):
    stations_in_quartier = stations[stations.geometry.within(quartier.geometry)]
    return stations_in_quartier

# Sélectionner le quartier
quartier_name = 'Javel 15Art' 
selected_quartier = quartiers[quartiers['district_name'] == quartier_name].iloc[0]

# Trouver les stations dans le quartier
stations_in_quartier = find_stations_in_quartier(stations, selected_quartier)

# Afficher les stations trouvées
print(stations_in_quartier)

# Créer une carte avec folium
m = folium.Map(location=[stations_in_quartier.geometry.y.mean(), stations_in_quartier.geometry.x.mean()], zoom_start=14)
folium.GeoJson(selected_quartier.geometry).add_to(m)

# Ajouter uniquement les stations dans le quartier
for idx, row in stations_in_quartier.iterrows():
    folium.Marker([row.geometry.y, row.geometry.x], popup=row['name']).add_to(m)

# Afficher la carte
m



                                           name  bike_stands  available_bikes  \
45                     15068 - BOULEVARD VICTOR           20                0   
134          15057 - BOSSOUTROT - MARTIAL VALIN           38                3   
185                              15042 - FARMAN           21                2   
237                              15056 - BALARD           22               22   
321            15108 - LOURMEL - MARIE SKOBTSOV           23               15   
331                                15120 - AVIA           29               15   
359   15104 - HÔPITAL GEORGES POMPIDOU (PROP 2)           65               24   
394                             15032 - LOURMEL           57               44   
396                    15058 - GEORGES POMPIDOU           46               27   
520                 15103 - SQUARE DES CÉVENNES           49               39   
601                             15059 - CITROEN           64               42   
686                         

## Afficher les station situées à 2km de la Tour Eiffel

In [4]:
from shapely.geometry import Point
import folium

# Définir les coordonnées gps de la Tour Eiffel
eiffel_tower_coords = (48.8586, 2.2845) 
eiffel_tower_point = Point(eiffel_tower_coords[1], eiffel_tower_coords[0]) 

# Fonction pour trouver les stations de vélo dans un rayon donné autour d'un point
def find_stations_within_radius(data, center, radius):
    buffer = center.buffer(radius)
    stations_within = data[data.geometry.within(buffer)]
    return stations_within

# Définir un rayon de 2 km
radius = 2000  # en mètres

# Trouver les stations dans le rayon de 2 km autour de la Tour Eiffel
stations_near_eiffel = find_stations_within_radius(stations, eiffel_tower_point, radius)

# Afficher les stations trouvées
print(stations_near_eiffel)

# Créer une carte avec folium
m = folium.Map(location=eiffel_tower_coords, zoom_start=14)
folium.Circle(eiffel_tower_coords, radius=radius, color='blue', fill=True).add_to(m)

# Ajouter uniquement les stations dans le rayon de 2 km
for idx, row in stations_near_eiffel.iterrows():
    folium.Marker([row.geometry.y, row.geometry.x], popup=row['name']).add_to(m)

# Afficher la carte
m


                                        name  bike_stands  available_bikes  \
0                     14002 - RASPAIL QUINET           44                4   
1        20503 - COURS DE VINCENNES PYRÉNÉES           21                3   
2                   20011 - PYRÉNÉES-DAGORNO           21                0   
3              31008 - VINCENNES (MONTREUIL)           56                0   
4                43006 - MINIMES (VINCENNES)           28               27   
...                                      ...          ...              ...   
1221                       12011 - CHARENTON           36               35   
1222             13110 - MOULIN DE LA POINTE           22                4   
1223                   07006 - QUAI VOLTAIRE           32               25   
1224  21311 - GAMBETTA (ISSY LES MOULINEAUX)           55               47   
1225                 01020 - RIVOLI CONCORDE           40               37   

                      geometry  
0      POINT (2.32956 48.8392)